In [67]:
import pandas as pd
import numpy as np
import psycopg2
import psycopg2.extras

In [49]:
df = pd.read_csv("./ecobicidata/ecobici_mar_resume.csv")
name_of_files = ["jan","feb","mar","apr","may","jun","jul"]
fulldf = pd.concat([pd.read_csv(f"./ecobicidata/ecobici_{x}.csv") for i,x in enumerate(name_of_files)])

In [50]:
def transform_df(ene = fulldf):
    ene["full_date_retiro"] = pd.to_datetime(ene["Fecha_Retiro"] + " " + ene["Hora_Retiro"], format="%d/%m/%Y %H:%M:%S").copy()
    ene["full_date_aribo"] = pd.to_datetime(ene["Fecha_Arribo"] + " " + ene["Hora_Arribo"], format="%d/%m/%Y %H:%M:%S").copy()
    ene["Mes"] = ene["full_date_retiro"].dt.month
    ene["Hora"] = ene["full_date_retiro"].dt.hour
    ene["time_delta"] = round((ene["full_date_aribo"]  - ene["full_date_retiro"]) / np.timedelta64(1,"m"),2)
    ene["Ciclo_Estacion_Retiro"]= ene[["Ciclo_Estacion_Retiro"]].astype(str)
    ene["Ciclo_Estacion_Retiro"] = [i[:-2] for i in ene["Ciclo_Estacion_Retiro"]]
    ene["Bici"]= ene[["Bici"]].astype(str)
    ene["Bici"] = [i[:-2] for i in ene["Bici"]]
    ene["viaje"] = ene["Ciclo_Estacion_Retiro"].astype(str)+"-"+ene["Ciclo_EstacionArribo"].astype(str)
    ene["Genero_Usuario"] = ene["Genero_Usuario"].fillna("X")
    ene = ene.dropna(axis=0).copy()
    return ene.iloc[:,[0,1,2,3,6,9,10,11,12,13,14]].sample(352301)

df = transform_df()

In [ ]:
df.to_csv("./ecobicidata/all_year.csv")

## Trips per Month

In [52]:
tripsperMonth = df[["Mes"]].groupby("Mes").size().reset_index()

In [53]:
tripsperMonth

,Mes,0
0,1.0,38509
1,2.0,43678
2,3.0,52770
3,4.0,52764
4,5.0,56657
5,6.0,55012
6,7.0,52903
7,12.0,8


## Most Popular Route

In [42]:
# df[["viaje","Mes"]].groupby(["Mes","viaje"]).size().reset_index().sort_values(by=0, ascending=False).pivot_table(columns="Mes",index="viaje", values=0, aggfunc="sum").sort_values(by=7, ascending=False).reset_index().head(100).fillna(0.0)

In [51]:
l = df[["viaje"]].groupby("viaje").size().reset_index().sort_values(by=0, ascending=False)

,viaje,0
54987,391-391,720
45327,33-33,304
31234,257-257,286
62516,449-449,248
31836,260-260,214
...,...,...
39759,30-233,1
39762,30-249,1
39773,30-26,1
39774,30-261,1


In [92]:
estaciones = pd.read_csv("./ecobicidata/estaciones-de-ecobici.csv",index_col=0).iloc[:,[0,1,3,4,5,8,9,10]]
estaciones["stationtype"] = [i.replace(",","") for i in estaciones["stationtype"].to_list()]
estaciones["zipcode"] = estaciones["zipcode"].fillna("0.0")
estaciones = estaciones.to_csv("./ecobicidata/estaciones_ecobici.csv")

In [96]:
dconnector="host=34.66.221.94 port=5432 dbname=ecobici user=postgres password=password"
nameOfTheFile = "estaciones_ecobici"

def new_ecobici_table(nf, dbConnection):
    conn = psycopg2.connect(dbConnection)
    cur = conn.cursor()
    cur.execute(f"""
        CREATE TABLE {nf}(
            id NUMERIC PRIMARY KEY,
            name text,
            address  text,
            zipcode float(6),
            districtcode VARCHAR(6),
            districtname text,
            location_lat float(8),
            location_lon float(8),
            stationtype text
        )
        """)

    conn.commit()
    print("Table Created.")

new_ecobici_table(nf=nameOfTheFile, dbConnection=dconnector)

Table Created.


In [97]:
def push_data_to_table(datefile,dbConnection):
    conn = psycopg2.connect(dbConnection)
    cur = conn.cursor()
    with open(f'./ecobicidata/{datefile}.csv', 'r', encoding="utf8") as f:
        # Notice that we don't need the `csv` module.
        next(f) # Skip the header row.
        cur.copy_from(f, f'{datefile}', sep=',')

    conn.commit()

push_data_to_table(datefile=nameOfTheFile,dbConnection=dconnector)